In [1]:
# Various pip
!pip install tensorflow
!pip install transformers
!pip install datasets
!pip install wordfreq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.3 MB/s eta 0:00:00


In [2]:
# Dataframe management
import pandas as pd

# Data manipulation
import numpy as np
import random

# NNs
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Transformers
import transformers

# Convert to HuggingFace dataset
import pyarrow as pa
from datasets import Dataset
import datasets

# Sklearn for train test split
from sklearn.model_selection import train_test_split

# Wordfreq to get the italian wordssss
from wordfreq import top_n_list

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [80]:
#PER FAVORE NON CANCELLATE :(
PATH_ELISA_MAIN_DATA = '/content/drive/My Drive/NLP_HATESPEECH/data/mainDataEN.csv'
PATH_ELISA_HASPEEDE2 = '/content/drive/My Drive/NLP_HATESPEECH/haspeede2_dev/haspeede2_dev_taskAB.tsv'
PATH_ELISA_MISOGYNY_DATA = '/content/drive/My Drive/NLP_HATESPEECH/misogyny_data/AMI2020_TrainingSet/AMI2020_training_raw.tsv'

In [104]:
# Get dataset
df_en = pd.read_csv(PATH_ELISA_MAIN_DATA)
df_ita_hs = pd.read_csv(PATH_ELISA_HASPEEDE2, sep='\t')
df_ita_m = pd.read_csv(PATH_ELISA_MISOGYNY_DATA, sep="\t")

In [105]:
df_ita_hs

,id,text,hs,stereotype
0,2066,"È terrorismo anche questo, per mettere in uno ...",0,0
1,2045,@user @user infatti finché ci hanno guadagnato...,0,0
2,61,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,0
3,1259,"@user ad uno ad uno, perché quando i migranti ...",0,0
4,949,Il divertimento del giorno? Trovare i patrioti...,0,0
...,...,...,...,...
6832,9340,Gli stati nazionali devono essere pronti a rin...,0,0
6833,9121,Il ministro dell'interno della Germania #Horst...,0,0
6834,8549,#Salvini: In Italia troppi si sono montati la ...,0,0
6835,9240,@user @user Chi giubila in buona fede non ha c...,0,0


In [106]:
def is_hate_speech(value):
  if value['class'] == 2:
    return 0 #no hate speech
  else:
     return 1 #hate speech

df_en['hate_speech'] = df_en.apply(is_hate_speech, axis=1)

df_en = df_en.drop(['Unnamed: 0','count', 'offensive_language', 'neither', 'class'], axis=1)

df_en.head()

,hate_speech,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [107]:
#stereotypes are considered as hate speech
df_ita_hs['hate_speech'] = df_ita_hs['stereotype'] | df_ita_hs['hs']
df_ita_hs = df_ita_hs.drop(['id', 'stereotype', 'hs'], axis=1)
df_ita_hs = df_ita_hs.rename(columns={'text ': 'text'}) #there was a space

#merge 'misogynous' and 'aggressiveness' column into a new hate speech column.
df_ita_m['hate_speech'] = df_ita_m['misogynous'] | df_ita_m['aggressiveness']
df_ita_m = df_ita_m.drop(['id', 'misogynous', 'aggressiveness'], axis=1)

In [85]:
df_ita_m

,text,hate_speech
0,@KassemAmin4 @Laylasexgdr Fatti trovare te lo...,1
1,@meb Tu dovresti ricominciare dai semafori a f...,1
2,"Amore,sei presentabile? Xchè così via Skype ti...",1
3,"@Il_nulla Salvo poi mandare la culona a Mosca,...",1
4,@GiorgiaMeloni @FratellidItaIia Vediamo Gentil...,1
...,...,...
4995,non ci posso credere sono queste le cose che m...,0
4996,sono così incazzata! dovete smetterla di consi...,0
4997,SENTITE PORCA PUTTANA TAE NON PUÒ POSTARE SUL ...,0
4998,ma vaffanculo senti che cazzo me li fai te i c...,0


In [108]:
# Concatenate the datasets vertically
df_ita = pd.concat([df_ita_hs, df_ita_m], axis=0, ignore_index=True)

# Reset the index of the merged dataset
df_ita = df_ita.reset_index(drop=True)

# Output the merged dataset
df_ita.head()

,text,hate_speech
0,"È terrorismo anche questo, per mettere in uno ...",0
1,@user @user infatti finché ci hanno guadagnato...,0
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0
3,"@user ad uno ad uno, perché quando i migranti ...",0
4,Il divertimento del giorno? Trovare i patrioti...,0


In [109]:
df_ita.columns

Index(['text', 'hate_speech'], dtype='object')

In [110]:
len(df_ita)

11837

In [76]:
df_ita.head()

,text,hate_speech,text
0,"È terrorismo anche questo, per mettere in uno ...",0,NaN
1,@user @user infatti finché ci hanno guadagnato...,0,NaN
2,"Corriere: Tangenti, Mafia Capitale dimenticata...",0,NaN
3,"@user ad uno ad uno, perché quando i migranti ...",0,NaN
4,Il divertimento del giorno? Trovare i patrioti...,0,NaN


In [73]:
df_ita_m.head()

,text,hate_speech
0,@KassemAmin4 @Laylasexgdr Fatti trovare te lo...,1
1,@meb Tu dovresti ricominciare dai semafori a f...,1
2,"Amore,sei presentabile? Xchè così via Skype ti...",1
3,"@Il_nulla Salvo poi mandare la culona a Mosca,...",1
4,@GiorgiaMeloni @FratellidItaIia Vediamo Gentil...,1


In [115]:
#training data for the italian dataframe
train_ita, test_ita = train_test_split(df_ita, test_size=0.2)

X_train_ita = train_ita['text'].reset_index(drop=True)
Y_train_ita = train_ita['hate_speech'].reset_index(drop=True)

X_test_ita = test_ita['text'].reset_index(drop=True)
Y_test_ita = test_ita['hate_speech'].reset_index(drop=True)

In [112]:
train_en, test_en = train_test_split(df_en, test_size=0.2)

X_train = train_en['tweet'].reset_index(drop=True)
Y_train = train_en['hate_speech'].reset_index(drop=True)

X_test = test_en['tweet'].reset_index(drop=True)
Y_test = test_en['hate_speech'].reset_index(drop=True)

X_it = df_ita['text']
Y_it = df_ita['hate_speech']

# THE BIBLE I FOLLOW

https://github.com/nlptown/nlp-notebooks/blob/master/Multilingual%20Embeddings%20-%203.%20Transfer%20Learning.ipynb

---

Start by downloading set of embeddings from the MUSE library, are fastText Wikipedia supervised word embeddings.


In [14]:
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec -O wiki.en.vec
!wget https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec -O wiki.it.vec

--2023-06-30 09:15:09--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.en.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.65.39.117, 18.65.39.124, 18.65.39.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.65.39.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628614720 (599M) [text/plain]
Saving to: ‘wiki.en.vec’

wiki.en.vec         100%[===================>] 599.49M   253MB/s    in 2.4s    

2023-06-30 09:15:12 (253 MB/s) - ‘wiki.en.vec’ saved [628614720/628614720]

--2023-06-30 09:15:12--  https://dl.fbaipublicfiles.com/arrival/vectors/wiki.multi.it.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.65.39.117, 18.65.39.124, 18.65.39.115, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.65.39.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 628718043 (600M) [text/plain]
Saving to: ‘wiki.it.vec’

wiki.it.vec         100%[==============

In [116]:
import numpy as np

def load_vectors(embedding_file_path):
    print("Loading vectors from", embedding_file_path)
    embeddings = []
    word2id = {}
    with open(embedding_file_path, 'r', encoding='utf-8') as f:
        next(f)
        for i, line in enumerate(f):
            word, emb = line.rstrip().split(' ', 1)
            emb = np.fromstring(emb, sep=' ')
            assert word not in word2id, 'word found twice'
            embeddings.append(emb)
            word2id[word] = len(word2id)

    embeddings = np.vstack(embeddings)
    return embeddings, word2id

# Produce the:
#   - embeddings_en → list of the embeddings of the words in wiki
#   - embedding_word2id_en → dictionary with each word and relative index in the embedding list
embeddings_en, embedding_word2id_en = load_vectors("wiki.en.vec")
embeddings_it, embedding_word2id_it = load_vectors("wiki.it.vec")

Loading vectors from wiki.en.vec
Loading vectors from wiki.it.vec


In [119]:
embeddings_en

array([[-0.0112864 , -0.00206967, -0.0515041 , ...,  0.0435525 ,
        -0.00774608,  0.0724234 ],
       [-0.0469256 , -0.00058526, -0.0750844 , ...,  0.0267626 ,
        -0.0513555 ,  0.0166127 ],
       [-0.0324474 , -0.0462027 , -0.00872643, ...,  0.0826505 ,
        -0.0649553 ,  0.0175795 ],
       ...,
       [-0.0608303 , -0.0172956 , -0.101635  , ...,  0.102298  ,
         0.0479459 ,  0.0387411 ],
       [-0.0867473 , -0.00469176, -0.104337  , ..., -0.00478122,
         0.130826  , -0.0207968 ],
       [-0.0119003 , -0.0123243 , -0.065963  , ...,  0.0184328 ,
        -0.0295059 , -0.0160871 ]])

In [120]:
embedding_word2id_en

{',': 0,
 '.': 1,
 'the': 2,
 '</s>': 3,
 'of': 4,
 '-': 5,
 'in': 6,
 'and': 7,
 "'": 8,
 ')': 9,
 '(': 10,
 'to': 11,
 'a': 12,
 'is': 13,
 'was': 14,
 'on': 15,
 's': 16,
 'for': 17,
 'as': 18,
 'by': 19,
 'that': 20,
 'it': 21,
 'with': 22,
 'from': 23,
 'at': 24,
 'he': 25,
 'this': 26,
 'be': 27,
 'i': 28,
 'an': 29,
 'utc': 30,
 'his': 31,
 'not': 32,
 '–': 33,
 'are': 34,
 'or': 35,
 'talk': 36,
 'which': 37,
 'also': 38,
 'has': 39,
 'were': 40,
 'but': 41,
 'have': 42,
 '#': 43,
 'one': 44,
 'rd': 45,
 'new': 46,
 'first': 47,
 'page': 48,
 'no': 49,
 'you': 50,
 'they': 51,
 'had': 52,
 'article': 53,
 't': 54,
 'who': 55,
 '?': 56,
 'all': 57,
 'their': 58,
 'there': 59,
 'been': 60,
 'made': 61,
 'its': 62,
 'people': 63,
 'may': 64,
 'after': 65,
 '%': 66,
 'other': 67,
 'should': 68,
 'two': 69,
 'score': 70,
 'her': 71,
 'can': 72,
 'would': 73,
 'more': 74,
 'if': 75,
 'she': 76,
 'about': 77,
 'when': 78,
 'time': 79,
 'team': 80,
 'american': 81,
 'such': 82,
 'th': 

In [121]:
# Deinfe some variables
VOCABULARY_SIZE = 10000
EMBEDDING_DIM = 300
SEQ_LENGTH = 300

INDEX_FROM = 3

In [122]:
X_train_ita = ['<START> ' + x for x in X_train_ita]
X_test_ita = ['<START> ' + x for x in X_test_ita]

In [123]:
X_train = ['<START> ' + x for x in X_train]
X_test= ['<START> ' + x for x in X_test]

In [128]:
# create the tokenizer
t = Tokenizer(oov_token="<unk>", num_words=VOCABULARY_SIZE+INDEX_FROM-1, filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')

# fit the tokenizer on the documents
#t.fit_on_texts(X_train)
t.fit_on_texts(X_train_ita)

t.word_index['<pad>'] = 0
t.index_word[0] = '<pad>'

# summarize what was learned
#print(t.word_counts)
#print(t.document_count)
print(t.word_index)
print(t.word_docs)

# integer encode documents
X_train_ita = t.texts_to_sequences(X_train_ita)
X_test_ita = t.texts_to_sequences(X_test_ita)

#X_train = t.texts_to_sequences(X_train)
#X_test = t.texts_to_sequences(X_test)

{'<unk>': 1, '<start>': 2, 'di': 3, 'e': 4, 'che': 5, 'user': 6, 'a': 7, 'la': 8, 'non': 9, 'url': 10, 'i': 11, 'il': 12, 'in': 13, 'per': 14, 'è': 15, 'un': 16, 'ma': 17, 'puttana': 18, 'porca': 19, 'sono': 20, 'da': 21, 'una': 22, 'con': 23, 'le': 24, 'rom': 25, 'si': 26, 'gli': 27, 'migranti': 28, 'se': 29, 'della': 30, 'tutti': 31, 'cazzo': 32, 'mi': 33, 'del': 34, 'immigrati': 35, 'ha': 36, 'come': 37, 'ti': 38, 'ci': 39, 'anche': 40, 'al': 41, 'lo': 42, 'dei': 43, 'co': 44, 't': 45, 'https': 46, 'io': 47, 'italia': 48, 'loro': 49, 'alla': 50, 'più': 51, 'roma': 52, 'solo': 53, 'italiani': 54, 'hanno': 55, 'perché': 56, 'sei': 57, 'o': 58, 'chi': 59, 'ho': 60, 'troia': 61, 'fare': 62, 'tutto': 63, 'casa': 64, 'poi': 65, 'li': 66, 'ai': 67, 'questo': 68, 'stranieri': 69, 'nel': 70, 'no': 71, 'noi': 72, 'culo': 73, 'fa': 74, 'figa': 75, 'cosa': 76, 'via': 77, 'culona': 78, 'quando': 79, 'fatto': 80, 'tettona': 81, 'ora': 82, 'questi': 83, 'ne': 84, 'questa': 85, 'campo': 86, 'salvin

In [129]:
#for italian
X_train_ita = tf.keras.utils.pad_sequences(X_train_ita, maxlen=SEQ_LENGTH)
X_test_ita = tf.keras.utils.pad_sequences(X_test_ita, maxlen=SEQ_LENGTH)

In [130]:
#for english
X_train = tf.keras.utils.pad_sequences(X_train, maxlen=SEQ_LENGTH)
X_test = tf.keras.utils.pad_sequences(X_test, maxlen=SEQ_LENGTH)

In [131]:
len(X_train[76])

300

In [132]:
def create_embedding_matrix(target_word2id, embedding_word2id, embeddings, num_rows, num_columns):
    embedding_matrix = np.zeros((num_rows, num_columns))
    for word, i in target_word2id.items():
        if i >= num_rows:
            break
        if word in embedding_word2id:
            embedding_matrix[i] = embeddings[embedding_word2id[word]]
    return embedding_matrix

In [135]:
word2id_en = t.word_index
word2id_it = t.word_index

embedding_matrix_it = create_embedding_matrix(word2id_it, embedding_word2id_it, embeddings_it, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)
embedding_matrix_en = create_embedding_matrix(word2id_en, embedding_word2id_en, embeddings_en, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)

In [136]:
embedding_matrix_it

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.0327415 ,  0.0348971 ,  0.0492003 , ...,  0.00657046,
         0.0301103 , -0.0277546 ],
       [-0.00980601, -0.0575472 , -0.131385  , ...,  0.0591859 ,
        -0.0313486 ,  0.112054  ],
       [-0.0657439 ,  0.00624939, -0.010534  , ...,  0.0128239 ,
        -0.0235025 , -0.0129116 ]])

In [25]:
embedding_matrix_en

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.0166294 ,  0.0186813 , -0.0765765 , ...,  0.0291019 ,
         0.059384  ,  0.0438444 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.00678915,  0.03004   , -0.00389318, ...,  0.11759   ,
         0.0913263 , -0.0269946 ]])

In [138]:
# Model
WEIGHTS = embedding_matrix_it #embedding_matrix_en
model = Sequential()
# ------
model.add(keras.layers.Embedding(VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM, input_length=SEQ_LENGTH, weights=[WEIGHTS], trainable=False))
# ------
model.add(keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(keras.layers.MaxPooling1D(pool_size=2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(250, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

# Compile
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 300, 300)          3000600   
                                                                 
 conv1d_2 (Conv1D)           (None, 300, 32)           28832     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 150, 32)          0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 4800)              0         
                                                                 
 dense_4 (Dense)             (None, 250)               1200250   
                                                                 
 dense_5 (Dense)             (None, 1)                 251       
                                                      

In [139]:
Y_train

0        1
1        1
2        1
3        0
4        0
        ..
19821    1
19822    0
19823    1
19824    1
19825    1
Name: hate_speech, Length: 19826, dtype: int64

In [141]:
NUM_EPOCHS=20
BATCH_SIZE=64

earlystop = EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train_ita, Y_train_ita, validation_data=(X_test_ita, Y_test_ita),
          epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[earlystop])

# Final evaluation of the model
scores = model.evaluate(X_test_ita, Y_test_ita)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/20
148/148 [==============================] - 27s 176ms/step - loss: 0.5253 - accuracy: 0.7285 - val_loss: 0.5102 - val_accuracy: 0.7449
Epoch 2/20
148/148 [==============================] - 29s 194ms/step - loss: 0.4328 - accuracy: 0.7914 - val_loss: 0.4781 - val_accuracy: 0.7791
Epoch 3/20
148/148 [==============================] - 33s 222ms/step - loss: 0.3663 - accuracy: 0.8323 - val_loss: 0.5011 - val_accuracy: 0.7534
Epoch 4/20
74/74 [==============================] - 2s 22ms/step - loss: 0.5479 - accuracy: 0.7546
Accuracy: 75.46%


In [142]:
from sklearn.metrics import precision_score

#si la precisione è interessante perché vedi i TP -> ovvero le parole hate speech effettivamente classificate come tali
predicted_labels = model.predict(X_test_ita)
predicted_labels = (predicted_labels >= 0.5).astype(int)

# Calculate precision
precision = precision_score(Y_test_ita, predicted_labels)

print("Precision: %.2f%%" % (precision * 100))

74/74 [==============================] - 2s 21ms/step
Precision: 79.77%


In [167]:
subset = 10
ground_truth_subset = Y_test_ita[:subset]
prediction_subset = predicted_labels[:subset]
text_subset = test_ita['text'].iloc[:subset]  #Non penso faccia lo stesso subset di ground_truth_subset

for i in range (0, subset):
  #TODO VEDERE DEL TESTO
  print("Sample %d - Actual: %d, Predicted: %d" % (i+1, ground_truth_subset[i], prediction_subset[i]))

Sample 1 - Actual: 0, Predicted: 1
Sample 2 - Actual: 1, Predicted: 1
Sample 3 - Actual: 0, Predicted: 0
Sample 4 - Actual: 1, Predicted: 1
Sample 5 - Actual: 1, Predicted: 1
Sample 6 - Actual: 1, Predicted: 1
Sample 7 - Actual: 1, Predicted: 1
Sample 8 - Actual: 0, Predicted: 0
Sample 9 - Actual: 1, Predicted: 0
Sample 10 - Actual: 0, Predicted: 0


In [ ]:


print(test_en.iloc[id]['tweet'], " | ", test_en.iloc[id]['hate_speech'])
model.predict(tweet)

FORSE SAREBBE BENE CONTROLLARE LE PERCENTUALI DI PRECISION/RECALL/ECC...

NON VORREI CHE METTESSE TIPO TUTTO HATE APPENA VEDE UNA PAOACIA :(

In [28]:
id = 15
tweet = np.array([X_test[id]])

print(test_en.iloc[id]['tweet'], " | ", test_en.iloc[id]['hate_speech'])
model.predict(tweet)

Jihadi rehab needs to account for the unpleasant fact that it's the worst behaviors that *attract* participants. http://t.co/nJJGA7S5LQ  |  0
1/1 [==============================] - 0s 391ms/step


array([[0.49810755]], dtype=float32)

In [29]:
fake_tweet = "fuck you are a genious!!!!"
fake_tweet = t.texts_to_sequences([fake_tweet])
fake_tweet = tf.keras.utils.pad_sequences(fake_tweet, maxlen=SEQ_LENGTH)

model.predict(fake_tweet)

1/1 [==============================] - 0s 23ms/step


array([[0.5011682]], dtype=float32)

In [30]:
word2id_it = {word: idx+INDEX_FROM for idx, word in enumerate(top_n_list('it', VOCABULARY_SIZE))}
word2id_it["<pad>"] = 0
word2id_it["<start>"] = 2
word2id_it["<unk>"] = 1

embedding_matrix_it = create_embedding_matrix(word2id_it, embedding_word2id_it, embeddings_it, VOCABULARY_SIZE+INDEX_FROM-1, EMBEDDING_DIM)

In [31]:
model.layers[0].set_weights([embedding_matrix_it])

In [32]:
ita = t.texts_to_sequences(X_it)
ita = tf.keras.utils.pad_sequences(ita, maxlen=SEQ_LENGTH)

result_ita = model.predict(ita)

214/214 [==============================] - 6s 28ms/step


In [34]:
result_ita

array([[0.4996693 ],
       [0.49806848],
       [0.502648  ],
       ...,
       [0.49637657],
       [0.49555895],
       [0.495185  ]], dtype=float32)

In [33]:
pepo = []
for res in result_ita:
  if res[0] >= 0.5:
    pepo.append(1)
  else:
    pepo.append(0)

In [35]:
right = 0
hihi = 0
for i in range(len(pepo)):
  if Y_it[i] == pepo[i]:
    right += 1
    if pepo[i] == 1:
      hihi = i

res = right/len(pepo)
print("Percentuale accuracy con ita: ", res)

Percentuale accuracy con ita:  0.46994295743747255


In [ ]:
X_it[hihi]

'Basta andare a vedere come l\'islam hanno combinato il Libano;ex "Paradiso Terrestre" che in accoglienza Cittadinanza e Ricongiunzione Famigliare,in Pietà Cristiana concessero il diritto di voto agli immigratiBastò un lustro per vedersi buttare fuori da tutte le Istituzioni🔫😯 URL '

In [ ]:
fake_tweet = ""
fake_tweet = t.texts_to_sequences([fake_tweet])
fake_tweet = tf.keras.utils.pad_sequences(fake_tweet, maxlen=SEQ_LENGTH)

model.predict(fake_tweet)

1/1 [==============================] - 0s 25ms/step


array([[0.07078993]], dtype=float32)

alora

non funziona molto bene anche se ho seguito per bene tutto quello che fa il tipo

secondo me è proprio per differenze linguistiche, teoricamente con i pesi dell'embedding in inglese ha capito come classificare i tweet riconoscendo la presenza di certe parole, e di conseguenza la loro posizione nello spazio dell'embedding

cambiando l'embedding lui si aspetta di trovare parole brutte in certe posizioni spaziali con determinate relazioni che probabilmente non si preservano nello spazio italiano

nonostante i risultati sia in inglese che in ita non siano poi così male, secondo me non performa comunque bene, ma credo la motivazione sia 50/50 dati e modello

i dati non sono un gran chè (come sono classificati) e con il nostro modello semplice semplice secondo me scegle le label in maniera conservativa, sparando 1 appena trova una minima parolaccia

in italiano non ho la più pallida idea di cosa reputi parolaccia, puoi scrivere le peggio cose e non lo labellllerà come offesa, sus......